### Note LGBM Currently Best Performer


In [1]:
from SimulateDay import stock_market_simulation, scale_data,train_Optimal_Action, get_stock_data, train_Optimal_Action, scale_and_obtain_data, fixFuckUp
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np 
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import pandas as pd
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import altair as alt

In [33]:
def importModels(symbol):
    specificModel = joblib.load(f"models/{symbol}_model.pkl")
    noCovidModel = train_Optimal_Action(symbol=symbol, action_column='Action')
    generalModel = xgb.Booster()    
    generalModel.load_model('models/all_stocks_incremental_model.pkl') 
    return specificModel, generalModel, noCovidModel
X_train, X_test, y_train, y_test = scale_and_obtain_data('INTC')


Adding columns...
Splitting data...


In [83]:
def train_classification_models_basic(X_train, y_train,symbol):
    models = {
    'Logistic Regression': LogisticRegression(random_state=42), # 0
    'Decision Tree': DecisionTreeClassifier(random_state=42), # 1
    'Random Forest': RandomForestClassifier(random_state=42), # 2
    'Gradient Boosting': GradientBoostingClassifier(random_state=42), # 3
    'SVM': SVC(), # 4
    'K-Nearest Neighbors': KNeighborsClassifier(), # 5
    'Neural Network': MLPClassifier(random_state=42), # 6
    'XGBoost': XGBClassifier(random_state=42), # 7
    'CatBoost': CatBoostClassifier(random_state=42, verbose=0), # 8
    'LightGBM': LGBMClassifier(random_state=42, verbose=0), # 9
    'Extra Trees': ExtraTreesClassifier(random_state=42, verbose=0), # 10
    'AdaBoost': AdaBoostClassifier(random_state=42), # 11
    'Stacking': StackingClassifier( # 12
        estimators=[
            # Actaul best estimators include CatBoost,but unfortanetly due to Kaggle's limitations it cant be used.
            ('rfc', RandomForestClassifier(random_state=42)),
            ('lbg', LGBMClassifier(random_state=42, verbose=0)),
            ('xgb', XGBClassifier(random_state=42))
        ],
        final_estimator=LogisticRegression()
    ),
}
    
    model_list = []

    for name, model in models.items():
        print(f"Training {name}...")
        model.fit(X_train, y_train)
        model_list.append(model)
    return model_list


In [10]:
messupday = fixFuckUp('INTC').tail(1)
updated_stock_df= get_stock_data('INTC').tail(365)
updated_stock_df = pd.concat([messupday, updated_stock_df]).sort_values(by='Date')
updated_stock_df
        

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
3355,2023-05-03,AAPL,167.449997,167.449997,170.919998,167.160004,169.500000,65136000.0
3356,2023-05-04,AAPL,165.789993,165.789993,167.039993,164.309998,164.889999,81235400.0
3357,2023-05-05,AAPL,173.570007,173.570007,174.300003,170.759995,170.979996,113316400.0
3358,2023-05-08,AAPL,173.500000,173.500000,173.850006,172.110001,172.479996,55962800.0
3359,2023-05-09,AAPL,171.770004,171.770004,173.539993,171.600006,173.050003,45326900.0
...,...,...,...,...,...,...,...,...
3716,2024-10-09,AAPL,229.539993,229.539993,229.750000,224.830002,225.229996,33591100.0
3717,2024-10-10,AAPL,229.039993,229.039993,229.500000,227.169998,227.779999,28183500.0
3718,2024-10-11,AAPL,227.550003,227.550003,229.410004,227.339996,229.300003,31668000.0
3719,2024-10-14,AAPL,0.000000,231.300003,231.729996,228.600006,228.699997,39882100.0


In [16]:
def get_final_portfolio_values(df, model):
    # Group by 'Stock Name' and get the last row for each group
    final_values = df.groupby('Stock Name').apply(lambda x: x.iloc[-1])
    
    # Extract 'Stock Name' and 'Portfolio Value' columns
    result = final_values[['Stock Name', 'Portfolio Value','Shares Held']].reset_index(drop=True)
    try:
        result['Model Name'] = model.__class__.__name__
    except:
        pass
    return result

In [84]:
def train_and_evaluate_models(symbol,portfolio):
    X_train, X_test, y_train, y_test = scale_and_obtain_data(symbol)
    models = train_classification_models_basic(X_train, y_train,symbol=symbol)
    
    messupday = fixFuckUp(symbol).tail(1)
    updated_stock_df= get_stock_data(symbol).tail(365)
    updated_stock_df = pd.concat([messupday, updated_stock_df]).sort_values(by='Date')
    for model in models:
        portfolioValue,_ = stock_market_simulation(model, 10000,365, updated_stock_df)
        finalPortfolioValue = get_final_portfolio_values(portfolioValue, model)
        portfolio = pd.concat([portfolio,finalPortfolioValue])
    portfolio['Total Profit %'] = (portfolio['Portfolio Value'] - 10000) / 10000 * 100
    chart = alt.Chart(portfolio).mark_bar().encode(
    x=alt.X('Stock Name:O', title='Tested Stocks'),
    y=alt.Y('Total Profit %:Q', title='Total Profit %'),
    color='Model Name:N',
    column='Model Name:O'
    ).properties(
        width=100,
        height=500,
        title='Total Profit % by Model Over Year To date Simulation',
        center=True
    ).configure_axis(
        labelAngle=45
    ).display()
    return chart,portfolio


In [85]:

INTC_chart, INTC_portfolio = train_and_evaluate_models('INTC',portfolio=pd.DataFrame())


Adding columns...
Splitting data...
Training Logistic Regression...
Training Decision Tree...
Training Random Forest...
Training Gradient Boosting...
Training SVM...
Training K-Nearest Neighbors...
Training Neural Network...
Training XGBoost...
Training CatBoost...
Training LightGBM...
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

alt.Chart(...)

In [86]:
AAPL_chart, AAPL_portfolio = train_and_evaluate_models('AAPL',portfolio=pd.DataFrame())


Adding columns...
Splitting data...
Training Logistic Regression...
Training Decision Tree...
Training Random Forest...
Training Gradient Boosting...
Training SVM...
Training K-Nearest Neighbors...
Training Neural Network...
Training XGBoost...
Training CatBoost...
Training LightGBM...
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

alt.Chart(...)

In [87]:
AMD_chart, AMD_portfolio = train_and_evaluate_models('AMD',portfolio=pd.DataFrame())


Adding columns...
Splitting data...
Training Logistic Regression...
Training Decision Tree...
Training Random Forest...
Training Gradient Boosting...
Training SVM...
Training K-Nearest Neighbors...
Training Neural Network...
Training XGBoost...
Training CatBoost...
Training LightGBM...
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

alt.Chart(...)

In [90]:
TSLA_chart, TSLA_portfolio = train_and_evaluate_models('TSLA',portfolio=pd.DataFrame())


Adding columns...


ValueError: Input X contains infinity or a value too large for dtype('float64').

In [92]:
MOH_chart, MOH_portfolio = train_and_evaluate_models('MOH',portfolio=pd.DataFrame())


Adding columns...
Splitting data...
Training Logistic Regression...
Training Decision Tree...
Training Random Forest...
Training Gradient Boosting...
Training SVM...
Training K-Nearest Neighbors...
Training Neural Network...
Training XGBoost...
Training CatBoost...
Training LightGBM...
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

alt.Chart(...)

In [93]:
NFLX_chart, NFLX_portfolio = train_and_evaluate_models('NFLX',portfolio=pd.DataFrame())


Adding columns...
Splitting data...
Training Logistic Regression...
Training Decision Tree...
Training Random Forest...
Training Gradient Boosting...
Training SVM...
Training K-Nearest Neighbors...
Training Neural Network...
Training XGBoost...
Training CatBoost...
Training LightGBM...
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

alt.Chart(...)

In [94]:
final_portfolio = pd.concat([INTC_portfolio,AAPL_portfolio,AMD_portfolio,TSLA_portfolio,MOH_portfolio,NFLX_portfolio])
final_portfolio_chart = alt.Chart(final_portfolio).mark_bar().encode(
    x=alt.X('Stock Name:O', title='Tested Stocks'),
    y=alt.Y('Total Profit %:Q', title='Total Profit %'),
    color='Model Name:N',
    column='Model Name:O'
).properties(
    width=100,
    height=500,
    title='Total Profit % by Model Over Year To date Simulation',
    center=True
).configure_axis(
    labelAngle=45
).display()

alt.Chart(...)

In [5]:
import pandas as pd

MODELCOMPARISON = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'SVM', 'XGBoost', 'Specific Model','Stacking',
              'Logistic Regression', 'Random Forest', 'SVM', 'XGBoost', 'Specific Model','Stacking',
              'Logistic Regression', 'Random Forest', 'SVM', 'XGBoost', 'Specific Model','Stacking',
              'Logistic Regression', 'Random Forest', 'SVM', 'XGBoost', 'Specific Model','Stacking',
              'Logistic Regression', 'Random Forest', 'SVM', 'XGBoost', 'Specific Model','Stacking',
              'Logistic Regression', 'Random Forest', 'SVM', 'XGBoost', 'Specific Model','Stacking',],
    'Stock': ['AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL','AAPL',
              'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT',
              'INTC', 'INTC', 'INTC', 'INTC', 'INTC', 'INTC',
              'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG',
              'AMD', 'AMD', 'AMD', 'AMD', 'AMD', 'AMD',
              'TSLA', 'TSLA', 'TSLA', 'TSLA', 'TSLA', 'TSLA'],
    'Final Portfolio Value': [12010.10, 12438.13, 10991.26, 12404.69, 12404.69, 12560.38,
                              11677.37, 11187.02, 11803.34, 11808.35, 11774.20, 10278.36,
                              9921.06, 9981.07, 9822.93, 9981.07, 9252.78,9980.80,
                              9969.69, 9944.17, 9964.88, 9956.80, 11123.06, 9986.28,
                              10187.84, 9921.48, 10446.35, 9921.48, 10742.80,9920.74,
                              9144.12, 10501.23, 9193.88, 10614.61, 9016.84,10897.54],
    'Total Profit %': [20.01, 24.38, 9.91, 24.04, 24.04, 25.60,
                       16.77, 11.87, 18.03, 18.08, 17.74, 2.78,
                       -0.79, 0.81, -1.77, 0.81, -7.47, -0.20,
                       -0.30, -0.56, -0.35, -0.44, 11.23, -0.14,
                       1.19, -0.79, 4.46, -0.79, 7.43, -0.80,
                       -8.56, 5.01, -8.06, 6.14, -9.83, 8.98]
}).sort_values('Final Portfolio Value')
# MODELCOMPARISON.to_csv('data/MODELCOMPARISON.csv')

In [8]:
sim_results = pd.read_csv('simResults/sim_results.csv')
sim_results['Stock Name'].unique()
model = LGBMClassifier(random_state=42, verbose=0)
for stock in sim_results['Stock Name'].unique():
    stock_df = scale_and_obtain_data(stock)
    model.fit(stock_df[0], stock_df[2])
    joblib.dump(model, f'LGBMmodels/{stock}_model.pkl')
    print(f"Model for {stock} saved successfully")
    


Adding columns...
Splitting data...
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [2]:
X_train, X_test, y_train, y_test = scale_and_obtain_data('TSLA')

Adding columns...
Splitting data...


In [7]:
y_train.value_counts()

Action
0    1313
1     894
2     769
Name: count, dtype: int64